# Data Pipeline
This notebook consolidates all data processing, Similarity computation, and exporting data to website format scripts into a single pipeline:
1. Generate and Export sequences from dynamic events for all matches (export_sequences.py)
2. Extract ball positions for all sequences (extract_Sequences_ball_postions.py)
3. Normalize attacking direction (Positions_Data_converter.py)
4. Similarity Score Calculation (Similarity_score.ipynb) 
5. Export the generated data to the format required by website for visualization (export_data_to_website.py)

## 1. Imports and Setup

In [ ]:
# Standard library imports
import pandas as pd
import json
import os
from pathlib import Path
from sklearn.preprocessing import  MinMaxScaler
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from datetime import datetime
# Import helper functions from processing scripts
from src.extract_Sequences_ball_positions import  extract_frame_numbers_from_sequence


# Configuration
MATCH_IDS = [1886347, 1899585, 1925299, 1953632, 1996435, 2006229, 2011166, 2013725, 2015213, 2017461]
DATA_DIR = os.path.join(os.getcwd(), 'data')
OUTPUT_DIR = os.path.join(os.getcwd(), 'output_json')

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Setup complete!")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Number of matches to process: {len(MATCH_IDS)}")

Setup complete!
Data directory: d:\college\pysport analytics cup\data
Output directory: d:\college\pysport analytics cup\trial_output_json
Number of matches to process: 10


## 2. Export Sequences from Dynamic Events for ALL Matches
This cell processes dynamic events data and groups them into sequences by team possession for all matches.

In [ ]:
def export_sequences_for_match(match_id):
    """
    Export sequences from dynamic events for a single match.
    Returns: (sequences, filtered_dataframe)
    """
    print(f"\n{'='*60}")
    print(f"Processing Match ID: {match_id}")
    print(f"{'='*60}")
    
    # Load dynamic events data
    url = f'https://raw.githubusercontent.com/SkillCorner/opendata/d276a0901fbe80b4790396b9bac93c0bdfaf694a/data/matches/{match_id}/{match_id}_dynamic_events.csv'
    local_path = os.path.join(DATA_DIR, f'{match_id}_dynamic_events.csv')
    
    if os.path.exists(local_path):
        print(f'✓ Reading events from local file: {local_path}')
        de_match = pd.read_csv(local_path)
    else:
        print(f'⚠ Downloading events from URL: {url}')
        de_match = pd.read_csv(url)
    
    # Filter out 'on_ball_engagement' events
    filtered_initial = de_match[de_match['event_type'] != 'on_ball_engagement'].copy()
    filtered_initial = filtered_initial.sort_values(['frame_start', 'frame_end']).reset_index(drop=True)
    
    print(f"Events after filtering: {len(filtered_initial)}")
    
    # Build sequences
    sequences = []
    current = None
    seq_counter = 1
    seq_ids = []
    included_idx = []
    off_run_included = 0
    off_run_excluded = 0
    
    for idx, row in filtered_initial.iterrows():
        team = row.get('team_id')
        ev_type = row.get('event_type')
        event = {
            'event_id': row.get('event_id'),
            'player_id': row.get('player_id'),
            'frame_start': int(row['frame_start']) if not pd.isna(row.get('frame_start')) else None,
            'frame_end': int(row['frame_end']) if not pd.isna(row.get('frame_end')) else None,
        }
        
        # Start new sequence if no current sequence
        if current is None:
            current = {
                'sequence_id': seq_counter,
                'team_id': team,
                'events': [event],
                'attacking_side': row.get('attacking_side')
            }
            seq_ids.append(seq_counter)
            included_idx.append(idx)
            seq_counter += 1
            if ev_type == 'off_ball_run':
                off_run_included += 1
            continue
        
        # Handle off_ball_run: include only if same team
        if ev_type == 'off_ball_run' and team != current['team_id']:
            off_run_excluded += 1
            continue
        
        # Group by team
        if team == current['team_id']:
            current['events'].append(event)
            seq_ids.append(current['sequence_id'])
            included_idx.append(idx)
            if ev_type == 'off_ball_run':
                off_run_included += 1
        else:
            sequences.append(current)
            current = {
                'sequence_id': seq_counter,
                'team_id': team,
                'events': [event],
                'attacking_side': row.get('attacking_side')
            }
            seq_ids.append(seq_counter)
            included_idx.append(idx)
            seq_counter += 1
            if ev_type == 'off_ball_run':
                off_run_included += 1
    
    if current is not None:
        sequences.append(current)
    
    # Attach sequence IDs to filtered dataframe
    filtered = filtered_initial.loc[included_idx].copy().reset_index(drop=True)
    filtered['sequence_id'] = seq_ids
    
    # Save sequences to JSON
    output_file = os.path.join(OUTPUT_DIR, f'{match_id}_sequences_excluding_offball_onball.json')
    with open(output_file, 'w', encoding='utf-8') as fh:
        json.dump({'sequences': sequences}, fh, ensure_ascii=False, indent=2)
    
    print(f"\n✓ Exported {len(sequences)} sequences to {output_file}")
    print(f"  - Included events: {len(filtered)}")
    print(f"  - off_ball_run included: {off_run_included}")
    print(f"  - off_ball_run excluded: {off_run_excluded}")
    
    return sequences, filtered

# Process ALL matches
print(f"\n{'#'*60}")
print(f"EXPORTING SEQUENCES FOR ALL {len(MATCH_IDS)} MATCHES")
print(f"{'#'*60}\n")

all_matches_sequences = {}
all_matches_filtered = {}
total_sequences = 0

for i, match_id in enumerate(MATCH_IDS, 1):
    print(f"\n[{i}/{len(MATCH_IDS)}] Processing match {match_id}...")
    try:
        sequences, filtered = export_sequences_for_match(match_id)
        all_matches_sequences[match_id] = sequences
        all_matches_filtered[match_id] = filtered
        total_sequences += len(sequences)
        print(f"✓ Match {match_id} complete!")
    except Exception as e:
        print(f"✗ Error processing match {match_id}: {str(e)}")
        continue

print(f"\n\n{'#'*60}")
print("SEQUENCE EXPORT COMPLETE")
print(f"{'#'*60}")
print(f"Total matches processed: {len(all_matches_sequences)}")
print(f"Total sequences exported: {total_sequences}")
print(f"\nSample sequence from first match:")
first_match = list(all_matches_sequences.keys())[0]
print(json.dumps(all_matches_sequences[first_match][0], indent=2))

## 3. Extract Ball Positions for ALL Sequences
This cell extracts ball position data from tracking URLs for each sequence across all matches.

In [ ]:
def extract_ball_positions_for_match(match_id, sequences):
    """
    Extract ball positions for all sequences in a match by fetching from URL.
    Fetches tracking data ONCE and reuses it for all sequences.
    Returns: list of sequence position data
    """
    print(f"\n{'='*60}")
    print(f"Extracting Ball Positions for Match ID: {match_id}")
    print(f"{'='*60}")
    
    # Construct tracking URL
    tracking_url = f"https://media.githubusercontent.com/media/SkillCorner/opendata/master/data/matches/{match_id}/{match_id}_tracking_extrapolated.jsonl"
    positions_output_file = os.path.join(OUTPUT_DIR, f'{match_id}_sequences_positions.json')
    
    print(f"📡 Fetching tracking data from: {tracking_url}")
    print(f"⏳ This may take a moment (large file)...")
    
    # Fetch tracking data ONCE for all sequences
    try:
        tracking_data = pd.read_json(tracking_url, lines=True)
        print(f"✓ Tracking data loaded: {len(tracking_data)} frames")
    except Exception as e:
        print(f"✗ Error fetching tracking data: {str(e)}")
        return None
    
    all_seq_positions = []
    success_count = 0
    fail_count = 0
    
    for idx, seq in enumerate(sequences):
        if not isinstance(seq, dict):
            continue
        
        # Get sequence ID
        seq_id = None
        for k in ('sequence_id', 'id', 'uid', 'sequenceId'):
            if k in seq:
                seq_id = seq[k]
                break
        
        if seq_id is None:
            print(f"  ⚠ Sequence {idx} has no ID, skipping")
            fail_count += 1
            continue
        
        # Extract frames for this sequence
        frames = extract_frame_numbers_from_sequence(seq)
        
        if not frames:
            print(f"  ⚠ Sequence {seq_id} has no frames, skipping")
            fail_count += 1
            continue
        
        # Extract positions from pre-loaded tracking data
        positions = extract_positions_from_dataframe(tracking_data, frames)
        
        # Convert to serializable format
        positions_serializable = {str(k): list(v) for k, v in positions.items()}
        
        # Attach positions to sequence
        seq['positions'] = positions_serializable
        
        # Collect for aggregate file
        seq_out = {
            'sequence_id': seq_id,
            'team_id': seq.get('team_id'),
            'frames': frames,
            'positions': positions_serializable
        }
        # Add attacking_side if present in the input sequence
        if 'attacking_side' in seq:
            seq_out['attacking_side'] = seq['attacking_side']
        
        all_seq_positions.append(seq_out)
        
        success_count += 1
        
        # Progress update every 50 sequences
        if (idx + 1) % 50 == 0:
            print(f"  Processed {idx + 1}/{len(sequences)} sequences...")
    
    # Final progress if not already shown
    if len(sequences) % 50 != 0:
        print(f"  Processed {len(sequences)}/{len(sequences)} sequences")
    
    # Save positions to JSON
    output_path = Path(positions_output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as fh:
        json.dump({
            'total_sequences': len(all_seq_positions),
            'sequences': all_seq_positions
        }, fh, indent=2)
    
    print(f"\n✓ Saved positions for {len(all_seq_positions)} sequences to {positions_output_file}")
    print(f"  - Success: {success_count}")
    print(f"  - Failed: {fail_count}")
    
    return all_seq_positions


def extract_positions_from_dataframe(tracking_df, wanted_frames):
    """
    Extract ball positions from a pre-loaded tracking DataFrame.
    Returns: dict frame->(x,y,z) for wanted_frames
    """
    results = {}
    wanted = set(wanted_frames)
    
    for _, obj in tracking_df.iterrows():
        # Determine frame index
        frame = None
        for k in ('frame', 'frame_idx', 'frame_index'):
            if k in obj and pd.notnull(obj[k]):
                try:
                    frame = int(obj[k])
                except Exception:
                    try:
                        frame = int(float(obj[k]))
                    except Exception:
                        frame = None
                break
        
        if frame is None or frame not in wanted:
            continue
        
        # Extract ball positions
        bx = by = bz = None
        if 'ball_data' in obj and isinstance(obj['ball_data'], dict):
            bd = obj['ball_data']
            bx = bd.get('x')
            by = bd.get('y')
            bz = bd.get('z')
        elif 'ball' in obj and isinstance(obj['ball'], dict):
            b = obj['ball']
            bx = b.get('x')
            by = b.get('y')
            bz = b.get('z')
        
        # Normalize numeric types
        def _safe_float(v):
            try:
                return None if v is None else float(v)
            except Exception:
                return None
        
        bx = _safe_float(bx)
        by = _safe_float(by)
        bz = _safe_float(bz)
        
        results[frame] = (bx, by, bz)
        
        # Early exit if all frames found
        if len(results) == len(wanted):
            break
    
    # Mark missing frames explicitly
    for f in wanted:
        if f not in results:
            results[f] = (None, None, None)
    
    return results


# Extract ball positions for ALL matches
print(f"\n{'#'*60}")
print(f"EXTRACTING BALL POSITIONS FOR ALL MATCHES")
print(f"{'#'*60}\n")

all_matches_positions = {}
total_positions = 0
failed_matches = []

for i, match_id in enumerate(MATCH_IDS, 1):
    print(f"\n[{i}/{len(MATCH_IDS)}] Processing match {match_id}...")
    
    if match_id not in all_matches_sequences:
        print(f"⚠ No sequences found for match {match_id}, skipping positions")
        continue
    
    try:
        positions = extract_ball_positions_for_match(match_id, all_matches_sequences[match_id])
        
        if positions:
            all_matches_positions[match_id] = positions
            total_positions += len(positions)
            print(f"✓ Match {match_id} complete!")
        else:
            failed_matches.append(match_id)
    except Exception as e:
        print(f"✗ Error processing match {match_id}: {str(e)}")
        failed_matches.append(match_id)
        continue

print(f"\n\n{'#'*60}")
print("BALL POSITION EXTRACTION COMPLETE")
print(f"{'#'*60}")
print(f"Total matches with positions: {len(all_matches_positions)}")
print(f"Total sequences with positions: {total_positions}")
print(f"Failed matches: {len(failed_matches)}")

if failed_matches:
    print(f"\nFailed matches (tracking data unavailable or error): {failed_matches}")

if all_matches_positions:
    print(f"\nSample position data from first match:")
    first_match = list(all_matches_positions.keys())[0]
    print(json.dumps(all_matches_positions[first_match][0], indent=2)[:800] + "...")

## 4. Convert and Normalize Attacking direction for Tracking Data
This cell normalizes orientation for all sequences and exports the format required by Similarity Score algorithm.

In [ ]:

from src.Positions_Data_converter import (
    extract_ball_coordinates
)

print(f"\n{'#'*60}")
print(f"CONVERTING AND NORMALIZING EVENT DATA")
print(f"{'#'*60}\n")

# Configuration
FIXED_NUM_EVENTS = 10  # Number of events per sequence adjust it if you want to analyze larger/smaller sequences
EXTRACTED_DIR = os.path.join(OUTPUT_DIR, 'extracted_sequences')

print(f"Target events per sequence: {FIXED_NUM_EVENTS}")
print(f"Output directory: {EXTRACTED_DIR}")


position_files_exist = []
for match_id in MATCH_IDS:
    position_file = os.path.join(OUTPUT_DIR, f'{match_id}_sequences_positions.json')
    if os.path.exists(position_file):
        position_files_exist.append(str(match_id))

print(f"\nFound position files for {len(position_files_exist)} matches")

output_csv = extract_ball_coordinates(
    match_ids=position_files_exist,
    fixed_num_events=FIXED_NUM_EVENTS,
    input_dir=OUTPUT_DIR,
    output_dir=EXTRACTED_DIR
)

if output_csv:
    print(f"\n✓ Event data conversion complete!")
    print(f"  Output file: {output_csv}")
    
    # Display sample of extracted data
    df_sample = pd.read_csv(output_csv)
    print(f"\nSample of extracted data:")
    display(df_sample.head())
    print(f"\nDataset shape: {df_sample.shape}")
    print(f"Columns: {list(df_sample.columns)}")
    
    # Check if frame ranges file was created
    frame_range_file = os.path.join(EXTRACTED_DIR, f"all_matches_{FIXED_NUM_EVENTS}_frame_ranges.csv")
    if os.path.exists(frame_range_file):
        df_ranges = pd.read_csv(frame_range_file)
        print(f"\nFrame ranges file created:")
        print(f"  Path: {frame_range_file}")
        print(f"  Shape: {df_ranges.shape}")
        display(df_ranges.head())
else:
    print(f"\n✗ No data was extracted")


############################################################
CONVERTING AND NORMALIZING EVENT DATA
############################################################

Target events per sequence: 10
Output directory: d:\college\pysport analytics cup\trial_output_json\extracted_sequences

Found position files for 1 matches

Processing match 1886347...
  Found 212 sequences
  Extracted sequences: 172
Saved frame ranges: d:\college\pysport analytics cup\trial_output_json\extracted_sequences\all_matches_10_frame_ranges.csv

Saved combined data: d:\college\pysport analytics cup\trial_output_json\extracted_sequences\all_matches_10_positions_normalized.csv
Total sequences processed: 212
Sequences normalized (flipped): 105
Total extracted subsets: 172

✓ Event data conversion complete!
  Output file: d:\college\pysport analytics cup\trial_output_json\extracted_sequences\all_matches_10_positions_normalized.csv

Sample of extracted data:


,match_id,sequence_id,team_id,attacking_side,normalized,num_positions,coordinates_json
0,1886347,6,4177,right_to_left,True,10,"[{""x"": 3.2, ""y"": -21.8, ""z"": 1.69}, {""x"": 11.8..."
1,1886347,7_subset0,1805,left_to_right,False,10,"[{""x"": 29.1, ""y"": -26.61, ""z"": 0.42}, {""x"": 29..."
2,1886347,7_subset1,1805,left_to_right,False,10,"[{""x"": 6.74, ""y"": 9.8, ""z"": 0.27}, {""x"": 6.58,..."
3,1886347,7_subset2,1805,left_to_right,False,10,"[{""x"": -26.05, ""y"": -4.3, ""z"": 0.18}, {""x"": -2..."
4,1886347,13_subset0,1805,left_to_right,False,10,"[{""x"": 27.84, ""y"": 29.25, ""z"": 0.52}, {""x"": 34..."



Dataset shape: (172, 7)
Columns: ['match_id', 'sequence_id', 'team_id', 'attacking_side', 'normalized', 'num_positions', 'coordinates_json']

Frame ranges file created:
  Path: d:\college\pysport analytics cup\trial_output_json\extracted_sequences\all_matches_10_frame_ranges.csv
  Shape: (172, 4)


,match_id,sequence_id,start_frame,end_frame
0,1886347,6,526,755
1,1886347,7_subset0,990,1161
2,1886347,7_subset1,1185,1303
3,1886347,7_subset2,1304,1371
4,1886347,13_subset0,1731,1845


## Normalize Ball Coordinates

This cell loads the extracted ball trajectory data and normalizes all x, y, z coordinates to a [0, 1] range using MinMax scaling.  
Normalization ensures that all sequences are on a common scale, which is essential for accurate similarity calculations.

In [8]:

# ============================================================================
 #Data Loading and Normalization
# ============================================================================
df = pd.read_csv(output_csv)
print(f"Loaded {len(df)} sequences")
print(f"Columns: {df.columns.tolist()}")

# Parse the coordinates_json column
df['coordinates'] = df['coordinates_json'].apply(json.loads)

# Extract all x, y, z values from all sequences to fit the scaler
all_x, all_y, all_z = [], [], []
for coords_list in df['coordinates']:
    for coord in coords_list:
        all_x.append(coord['x'])
        all_y.append(coord['y'])
        all_z.append(coord.get('z', 0))

print(f"\nTotal coordinates collected: {len(all_x)}")
print(f"X range: [{min(all_x):.2f}, {max(all_x):.2f}]")
print(f"Y range: [{min(all_y):.2f}, {max(all_y):.2f}]")
print(f"Z range: [{min(all_z):.2f}, {max(all_z):.2f}]")

# Fit scalers on all coordinates
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_z = MinMaxScaler()

scaler_x.fit(np.array(all_x).reshape(-1, 1))
scaler_y.fit(np.array(all_y).reshape(-1, 1))
scaler_z.fit(np.array(all_z).reshape(-1, 1))

# Normalize each coordinate in each sequence
def normalize_coordinates(coords_list):
    normalized = []
    for coord in coords_list:
        norm_coord = {
            'x': scaler_x.transform([[coord['x']]])[0][0],
            'y': scaler_y.transform([[coord['y']]])[0][0],
            'z': scaler_z.transform([[coord.get('z', 0)]])[0][0]
        }
        normalized.append(norm_coord)
    return normalized

df['normalized_coordinates'] = df['coordinates'].apply(normalize_coordinates)

print("\nNormalization complete!")
print(df.head())


Loaded 172 sequences
Columns: ['match_id', 'sequence_id', 'team_id', 'attacking_side', 'normalized', 'num_positions', 'coordinates_json']

Total coordinates collected: 1720
X range: [-52.38, 49.36]
Y range: [-34.10, 33.86]
Z range: [-0.19, 7.31]

Normalization complete!
   match_id sequence_id  team_id attacking_side  normalized  num_positions  \
0   1886347           6     4177  right_to_left        True             10   
1   1886347   7_subset0     1805  left_to_right       False             10   
2   1886347   7_subset1     1805  left_to_right       False             10   
3   1886347   7_subset2     1805  left_to_right       False             10   
4   1886347  13_subset0     1805  left_to_right       False             10   

                                    coordinates_json  \
0  [{"x": 3.2, "y": -21.8, "z": 1.69}, {"x": 11.8...   
1  [{"x": 29.1, "y": -26.61, "z": 0.42}, {"x": 29...   
2  [{"x": 6.74, "y": 9.8, "z": 0.27}, {"x": 6.58,...   
3  [{"x": -26.05, "y": -4.3, "z": 0.

## Compute the DTW Distance Matrix for Ball Trajectories

This cell computes the pairwise similarity between all ball trajectory sequences using Dynamic Time Warping (DTW).  
It builds a distance matrix where each entry quantifies how similar two plays are, enabling the identification of similar ball movement patterns across all matches.

In [ ]:
# ============================================================================
 # DTW Distance Matrix Computation
# ============================================================================

# Get number of Events dynamically from the data
Number_of_events = int(df['num_positions'].iloc[0])
print(f"Number of Events per sequence: {Number_of_events}")



# Create sequence identifiers (matchid_sequenceid)
# sequence_id already contains subset info like "6_subset0"
df['sequence_identifier'] = df['match_id'].astype(str) + '_' + df['sequence_id'].astype(str)

# Extract base sequence (remove subset suffix)
def get_base_sequence(seq_id):
    """
    Extract base sequence from identifier.
    Examples:
    - "1886347_6_subset0" -> "1886347_6"
    - "1886347_6" -> "1886347_6"
    """
    if '_subset' in seq_id:
        return seq_id.rsplit('_subset', 1)[0]
    return seq_id

df['base_sequence'] = df['sequence_identifier'].apply(get_base_sequence)

# Convert normalized coordinates to numpy arrays
def coords_to_array(coords_list):
    return np.array([[c['x'], c['y'], c['z']] for c in coords_list])

df['coord_array'] = df['normalized_coordinates'].apply(coords_to_array)

# Display statistics
print(f"\n{'='*60}")
print("Dataset Statistics:")
print(f"{'='*60}")
print(f"Total sequences: {len(df)}")
subset_count = df['sequence_identifier'].str.contains('_subset').sum()
unique_bases = df['base_sequence'].nunique()
print(f"Sequences with subsets: {subset_count}")
print(f"Unique base sequences: {unique_bases}")
print(f"Average subsets per base: {len(df) / unique_bases:.1f}")
print(f"{'='*60}\n")

# Prepare data
n_sequences = len(df)
sequence_ids = df['sequence_identifier'].values
coord_arrays = df['coord_array'].values
base_sequences = df['base_sequence'].values

# Initialize distance matrix with NaN
dtw_matrix = np.full((n_sequences, n_sequences), np.nan)

print(f"Computing DTW distances for {n_sequences} sequences...")
total_possible = n_sequences * (n_sequences - 1) // 2
print(f"Total possible comparisons: {total_possible:,}")

# Compute DTW
computed = 0
skipped = 0
start_time = datetime.now()

for i in range(n_sequences):
    if i % 10 == 0 and i > 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = computed / elapsed if elapsed > 0 else 0
        remaining = (total_possible - computed - skipped) / rate if rate > 0 else 0
        print(f"Processing sequence {i+1}/{n_sequences}... (computed: {computed:,}, skipped: {skipped:,}, rate: {rate:.1f}/s, ETA: {remaining/3600:.1f}h)")
    
    for j in range(i + 1, n_sequences):
        # Skip comparisons between subsets of the same base sequence
        if base_sequences[i] == base_sequences[j]:
            skipped += 1
            continue
        
        # Compute DTW distance
        distance, _ = fastdtw(coord_arrays[i], coord_arrays[j], dist=euclidean)
        # Normalize by number of positions
        dtw_matrix[i, j] = distance / Number_of_events
        dtw_matrix[j, i] = distance / Number_of_events
        computed += 1

# Set diagonal to 0
np.fill_diagonal(dtw_matrix, 0)

print(f"\n{'='*60}")
print("DTW Computation Complete!")
print(f"{'='*60}")
print(f"Computed: {computed:,}")
print(f"Skipped: {skipped:,}")
print(f"Total time: {(datetime.now() - start_time).total_seconds()/3600:.2f} hours")
print(f"{'='*60}\n")

# Create DataFrame
dtw_df = pd.DataFrame(dtw_matrix, index=sequence_ids, columns=sequence_ids)
print(f"DTW Matrix shape: {dtw_df.shape}")

# Save DTW distance matrix to CSV
output_file = os.path.join(EXTRACTED_DIR, f'dtw_distance_matrix_{Number_of_events}_positions_normalized.csv')
dtw_df.to_csv(output_file)
print(f"✓ Saved DTW distance matrix to: {output_file}")

# Display sample
print(f"\nSample distances (first 5x5):")
print(dtw_df.iloc[:5, :5])

## Exporting Similarity Scores for Website Integration

This section exports the top N most similar plays for each match in the format required by the website.  
It loads the DTW distance matrix and frame data, ensures all necessary files are available, and saves the results as JSON for direct use in the web application.
*Hint it downloads tracking data to extract players postions for each sequence for visualization

In [ ]:

from src.export_data_to_website import (
    load_frames_lookup,
    load_dtw_matrix,
    process_match_sequences
)
from pathlib import Path
import os
from src.utility import download_match_files


TARGET_MATCH_ID = 1886347 # Change this to the desired match ID you want to find similar plays for
TOP_N = 5 # Change this to desired number of similar plays
DTW_CSV = os.path.join(EXTRACTED_DIR, f'dtw_distance_matrix_{Number_of_events}_positions_normalized.csv')
FRAMES_CSV = os.path.join(EXTRACTED_DIR, f'all_matches_{Number_of_events}_frame_ranges.csv')

TRACKING_DIR = r"./Data" #If data is already downloaded, specify the path here
METADATA_DIR = r"./Data"
Path(TRACKING_DIR).mkdir(parents=True, exist_ok=True)
Path(METADATA_DIR).mkdir(parents=True, exist_ok=True)
for match_id in MATCH_IDS:
    download_match_files(match_id, TRACKING_DIR, METADATA_DIR)

OUTPUT_DIR = Path(rf"./website/public/{TARGET_MATCH_ID}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print(f"Generating Similar Plays JSON for Match {TARGET_MATCH_ID}")
print("=" * 80)

# ============================
# LOAD DATA
# ============================

dtw_matrix = load_dtw_matrix(DTW_CSV)
frames_lookup = load_frames_lookup(FRAMES_CSV)

# ============================
# RUN PIPELINE
# ============================

process_match_sequences(
    target_match_id=TARGET_MATCH_ID,
    dtw_matrix=dtw_matrix,
    frames_lookup=frames_lookup,
    tracking_dir=TRACKING_DIR,
    metadata_dir=METADATA_DIR,
    output_dir=OUTPUT_DIR,
    top_n=TOP_N
)

print("=" * 80)
print("Done!")
print(f"JSON files saved to: {OUTPUT_DIR}")
print("=" * 80)


Generating Similar Plays JSON for Match 1886347
Loading DTW matrix from: d:\college\pysport analytics cup\output_json\extracted_sequences\dtw_distance_matrix_10_positions_normalized.csv
Loading Frames CSV from: d:\college\pysport analytics cup\output_json\extracted_sequences\all_matches_10_frame_ranges.csv

Found 172 sequences for match 1886347

Processing 1886347_6 (Frames: 526-755)...
Loading metadata from: D:\college\pysport analytics cup\Data\1886347_match.json
Home Team: Auckland FC (ID: 4177)
Away Team: Newcastle United Jets FC (ID: 1805)
Loading tracking data from: D:\college\pysport analytics cup\Data\1886347_tracking_extrapolated.jsonl
Looking for frames between 526 and 755
Loaded 230 tracking frames
  DEBUG: tracking_df shape = (5060, 12)
  DEBUG: tracking_df columns = ['frame', 'x', 'y', 'player_id', 'is_detected', 'ball_x', 'ball_y', 'ball_z', 'is_detected_ball', 'team_id', 'short_name', 'number']
  DEBUG: First row:
frame                    526
x                     -49.46

KeyboardInterrupt: 